### Relation Extraction on FewRel

In [1]:
import sys
sys.path.append('../')
import models
from utils.data_loader import JSONFileDataLoader
from utils.framework import FewShotREFramework, FewShotREModel
from models.sentence_encoding.basic_sentence_encoder import CNNSentenceEncoder
from models.proto_model import Proto
from torch import optim

#### Модель 0

Эта модель была описана в статье по FewRel. Она выбрана в качестве бейзлайна (так как не реализует методов few-shot learning). На 5-way, 5-shot варианте задачи получена точность 68.77 на тестовой выборке при использовании kNN.

#### Модель 1

Для начала попробуем повторить исследование, проведенное авторами статьи, при помощи Prototypical Networks. Для этого сделаем такое же векторное представление предложений, как и авторы: написано в `models/sentence_encoding/basic_sentence_encoder`

In [2]:
N = 5
K = 5

max_length = 40
train_data_loader = JSONFileDataLoader('../data/fewrel_train.json', '../data/wordvecs.json', max_length = max_length)

Pre-processed files exist. Loading them...
Finish loading


In [3]:
val_data_loader = JSONFileDataLoader('../data/fewrel_val.json', '../data/wordvecs.json', max_length=max_length)

Pre-processed files exist. Loading them...
Finish loading


In [4]:
framework = FewShotREFramework(train_data_loader, val_data_loader, None)
sentence_encoder = CNNSentenceEncoder(train_data_loader.word_vec_mat,
                                      max_length, word_embedding_dim = 300)

In [5]:
model = Proto(sentence_encoder)
framework.train(model, 'proto', 4, 20, N, K, 5, cuda = False)

Start training...


..\utils\framework.py:161: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(parameters_to_optimize, 10)


KeyboardInterrupt: 

In [6]:
# Checking logfiles
model_small = Proto(sentence_encoder)
framework.train(model_small, 'proto', 4, 20, N, K, 5, cuda = False, val_step = 3, val_iter = 3, train_iter = 9)

Start training...
step:    3 | loss: 3.042896, accuracy: 5.87%
[EVAL] step:    3 | accuracy: 33.00%
Best checkpoint
step:    6 | loss: 3.046019, accuracy: 4.75%
[EVAL] step:    3 | accuracy: 31.00%
step:    9 | loss: 3.035750, accuracy: 4.75%
[EVAL] step:    3 | accuracy: 35.33%
Best checkpoint

####################

Finish training proto

Successfully loaded checkpoint './checkpoint\proto.pth.tar'


AttributeError: 'NoneType' object has no attribute 'next_batch'